# Setup Environment

Import required packages:

In [ ]:
# Setup detectron2
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()

import os, cv2, csv, ntpath
import numpy as np
from matplotlib import pyplot as plt
from tqdm import tqdm

from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg

Global variables and settings:

In [ ]:
path_dir_project = "/thecube/students/jravagli"

# Paths USED datasets
path_dir_datasets = os.path.join(path_dir_project, "datasets/used")
path_train_dataset = os.path.join(path_dir_datasets, "train")
path_test_dataset = os.path.join(path_dir_datasets, "test")
path_train_part_1 = os.path.join(path_train_dataset, "train-part1")
path_train_part_2 = os.path.join(path_train_dataset, "train-part2")
path_test_part_1 = os.path.join(path_test_dataset, "test-part1")
path_test_part_2 = os.path.join(path_test_dataset, "test-part2")
# Path merged annotation files
path_train_ann = os.path.join(path_train_dataset, "train.txt")
path_test_ann = os.path.join(path_train_dataset, "test.txt")
# Paths preprocessed datasets
path_dir_pp_datasets = os.path.join(path_dir_project, "datasets/used-pp")
path_pp_train_dataset = os.path.join(path_dir_pp_datasets, "train")
path_pp_valid_dataset = os.path.join(path_dir_pp_datasets, "valid")
path_pp_test_dataset = os.path.join(path_dir_pp_datasets, "test")
# Path preprocessed annotation files
path_pp_train_ann = os.path.join(path_dir_pp_datasets, "train.txt")
path_pp_valid_ann = os.path.join(path_dir_pp_datasets, "valid.txt")
path_pp_test_ann = os.path.join(path_dir_pp_datasets, "test.txt")
# Paths model
path_model_dir = os.path.join(path_dir_project, "outputs/detectron-12-epochs")
path_model_weights = os.path.join(path_model_dir, "model_final.pth")

n_classes = 14

# Organize USED dataset

Define a function that read the annotation file of a single event class and append the data to a unique file. At the end of the elaboration this file will contain the path and the annotation of all images in the dataset:

In [ ]:
def process_annotation_file(path_class_ann, path_out_ann, class_id, path_imgs):
    print(f"Processing annotation file {path_class_ann}")
    image_class = class_id
    list_path_imgs = []
    with open(path_class_ann) as file:
        csv_reader = csv.reader(file, delimiter=' ')
        for row in tqdm(csv_reader):
            if '.jpg' not in row[0]:
                continue
            else:
                image_name = row[0]
                list_path_imgs.append(os.path.join(path_imgs, image_name))

    with open(path_out_ann, 'a') as file:
        writer = csv.writer(file)
        for item in list_path_imgs:
            writer.writerow([item, class_id])
    
    print(f"Done")

Process all the train annotation files to create a unique file with image paths and classes:

In [ ]:
# Clean old file
if os.path.isfile(path_train_ann):
    os.remove(path_train_ann)
    print(f"Old {path_train_ann} deleted")

# Process all the annotation files inside part 1
process_annotation_file(os.path.join(path_train_part_1, "concert_train.txt"), path_train_ann, 0, path_train_part_1)
process_annotation_file(os.path.join(path_train_part_1, "graduation_train.txt"), path_train_ann, 1, path_train_part_1)
process_annotation_file(os.path.join(path_train_part_1, "meeting_train.txt"), path_train_ann, 2, path_train_part_1)
process_annotation_file(os.path.join(path_train_part_1, "mountaintrip_train.txt"), path_train_ann, 3, path_train_part_1)
process_annotation_file(os.path.join(path_train_part_1, "picnic_train.txt"), path_train_ann, 4, path_train_part_1)
process_annotation_file(os.path.join(path_train_part_1, "sea_holiday_train.txt"), path_train_ann, 5, path_train_part_1)
process_annotation_file(os.path.join(path_train_part_1, "ski_holiday_train.txt"), path_train_ann, 6, path_train_part_1)
process_annotation_file(os.path.join(path_train_part_1, "wedding_train.txt"), path_train_ann, 7, path_train_part_1)
# Process all the annotation files inside part 2
process_annotation_file(os.path.join(path_train_part_2, "concert_train.txt"), path_train_ann, 0, path_train_part_2)
process_annotation_file(os.path.join(path_train_part_2, "conference_train.txt"), path_train_ann, 8, path_train_part_2)
process_annotation_file(os.path.join(path_train_part_2, "Exibition_train.txt"), path_train_ann, 9, path_train_part_2)
process_annotation_file(os.path.join(path_train_part_2, "fashion_train.txt"), path_train_ann, 10, path_train_part_2)
process_annotation_file(os.path.join(path_train_part_2, "protest_train.txt"), path_train_ann, 11, path_train_part_2)
process_annotation_file(os.path.join(path_train_part_2, "sport_train.txt"), path_train_ann, 12, path_train_part_2)
process_annotation_file(os.path.join(path_train_part_2, "theaterdance_train.txt"), path_train_ann, 13, path_train_part_2)

Process all the test annotation files to create a unique file with image paths and classes:

In [ ]:
# Clean old file
if os.path.isfile(path_test_ann):
    os.remove(path_test_ann)
    print(f"Old {path_test_ann} deleted")

# Process all the annotation files inside part 1
process_annotation_file(os.path.join(path_test_part_1, "concert_test.txt"), path_test_ann, 0, path_test_part_1)
process_annotation_file(os.path.join(path_test_part_1, "graduation_test.txt"), path_test_ann, 1, path_test_part_1)
process_annotation_file(os.path.join(path_test_part_1, "meeting_test.txt"), path_test_ann, 2, path_test_part_1)
process_annotation_file(os.path.join(path_test_part_1, "mountaintrip_test.txt"), path_test_ann, 3, path_test_part_1)
process_annotation_file(os.path.join(path_test_part_1, "picnic_test.txt"), path_test_ann, 4, path_test_part_1)
process_annotation_file(os.path.join(path_test_part_1, "sea_holiday_test.txt"), path_test_ann, 5, path_test_part_1)
process_annotation_file(os.path.join(path_test_part_1, "ski_holiday_test.txt"), path_test_ann, 6, path_test_part_1)
process_annotation_file(os.path.join(path_test_part_1, "wedding_test.txt"), path_test_ann, 7, path_test_part_1)
# Process all the annotation files inside part 2
process_annotation_file(os.path.join(path_test_part_2, "concert_test.txt"), path_test_ann, 0, path_test_part_2)
process_annotation_file(os.path.join(path_test_part_2, "conference_test.txt"), path_test_ann, 8, path_test_part_2)
process_annotation_file(os.path.join(path_test_part_2, "Exibition_test.txt"), path_test_ann, 9, path_test_part_2)
process_annotation_file(os.path.join(path_test_part_2, "fashion_test.txt"), path_test_ann, 10, path_test_part_2)
process_annotation_file(os.path.join(path_test_part_2, "protest_test.txt"), path_test_ann, 11, path_test_part_2)
process_annotation_file(os.path.join(path_test_part_2, "sport_test.txt"), path_test_ann, 12, path_test_part_2)
process_annotation_file(os.path.join(path_test_part_2, "theaterdance_test.txt"), path_test_ann, 13, path_test_part_2)

# Preprocess USED dataset

Use the trained Mask R-CNN network for detecting clothes in the USED images. We create a new dataset containing the USED images with the clothes isolated from the background.

Load the Mask R-CNN network:

In [ ]:
# Use the same configuration used for training, with some changes for the 
# evaluation
cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml"))
cfg.DATASETS.TRAIN = ()
cfg.DATASETS.TEST = ()
cfg.DATALOADER.NUM_WORKERS = 2
cfg.MODEL.WEIGHTS = path_model_weights
cfg.SOLVER.IMS_PER_BATCH = 8
cfg.SOLVER.BASE_LR = 0.00025  # pick a good LR
cfg.SOLVER.MAX_ITER = 300    # 300 iterations seems good enough for this toy dataset; you will need to train longer for a practical dataset
cfg.SOLVER.STEPS = []        # do not decay learning rate
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 128   # faster, and good enough for this toy dataset (default: 512)
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 13
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.7   # Confidence threshold for predictions

predictor = DefaultPredictor(cfg)

Function to load the images names and class from the txt files of the annotations:

In [ ]:
def read_annotation_file(path_file):
    with open(path_file) as f:
        list_lines = f.readlines()
    
    list_images_names = []
    list_images_classes = []
    for line in list_lines:
        splits = line.split(",")
        list_images_names.append(splits[0].strip())
        list_images_classes.append(splits[1].strip())
    
    return list_images_names, list_images_classes

Function to preprocess and save the train and validation datasets:

In [ ]:
def preprocess_train_dataset(path_train, path_train_ann, path_pp_train, path_pp_train_ann, path_pp_valid, path_pp_valid_ann):
    list_path_images, list_images_classes = read_annotation_file(path_train_ann)

    # Read and preprocess all the images inside the dataset
    list_pp_train_paths = []
    list_pp_train_classes = []
    list_pp_train_clothes = []
    list_pp_valid_paths = []
    list_pp_valid_classes = []
    list_pp_valid_clothes = []
    for i in tqdm(range(len(list_path_images))):
        path_img = list_path_images[i]
        img_class = list_images_classes[i]
        img_name = ntpath.basename(path_img)

        img = cv2.imread(path_img)
        if img is not None:
            res = predictor(img)
            instances = res["instances"].to("cpu")

            # Check if the model found clothes in the image
            if len(instances):
                # About 15% of the images are assigned to the validation dataset
                assign_to_valid = np.random.rand() < 0.15
                if assign_to_valid:
                    path_pp_dataset = path_pp_valid
                    list_pp_paths = list_pp_valid_paths
                    list_pp_classes = list_pp_valid_classes
                    list_pp_clothes = list_pp_valid_clothes
                else:
                    path_pp_dataset = path_pp_train
                    list_pp_paths = list_pp_train_paths
                    list_pp_classes = list_pp_train_classes
                    list_pp_clothes = list_pp_train_clothes

                # Use the instance segmentation masks to crop the clothes area
                for pred_mask, pred_box, pred_clothes, i in zip(instances.pred_masks,instances.pred_boxes,
                                                                  instances.pred_classes.numpy(), range(1,len(instances))):
                    res_img = np.repeat(pred_mask.numpy()[:, :, None], 3, axis=-1)*img
                    w = int(pred_box[2] - pred_box[0])
                    h = int(pred_box[3] - pred_box[1])
                    y = int(pred_box[1])
                    x = int(pred_box[0])
                    crop_img = res_img[y:y+h, x:x+w]

                    # Save the preprocessed image in the folder dedicated to its class
                    path_pp = os.path.join(path_pp_dataset, img_class, os.path.splitext(img_name)[0] + f"-{i}.jpg")
                    list_pp_paths.append(path_pp)
                    list_pp_classes.append(img_class)
                    list_pp_clothes.append(pred_clothes) # Clothes class predicted by mask-r-cnn

                    cv2.imwrite(path_pp, crop_img)

    # Save the train annotation file
    file_string = ""
    for i in range(len(list_pp_train_paths)):
        file_string += list_pp_train_paths[i] + "," + list_pp_train_classes[i] + "," + str(list_pp_train_clothes[i]) + "\n"

    with open(path_pp_train_ann, "w") as f:
        f.write(file_string)
        
    # Save the validation annotation file
    file_string = ""
    for i in range(len(list_pp_valid_paths)):
        file_string += list_pp_valid_paths[i] + "," + list_pp_valid_classes[i] + "," + str(list_pp_valid_clothes[i]) + "\n"

    with open(path_pp_valid_ann, "w") as f:
        f.write(file_string)

Function to preprocess and save the test dataset:

In [ ]:
def preprocess_test_dataset(path_dataset, path_ann, path_pp_dataset, path_pp_ann):
    list_path_images, list_images_classes = read_annotation_file(path_ann)

    # Read and preprocess all the images inside the dataset
    list_pp_paths = []
    list_pp_classes = []
    list_pp_clothes = []
    for i in tqdm(range(len(list_path_images))):
        path_img = list_path_images[i]
        img_class = list_images_classes[i]
        img_name = ntpath.basename(path_img)
        
        img = cv2.imread(path_img)
        if img is not None:
            res = predictor(img)
            instances = res["instances"].to("cpu")

            # Check if the model found clothes in the image
            if len(instances):
                # Use the instance segmentation masks to crop the clothes area
                for pred_mask, pred_box, pred_clothes, i in zip(instances.pred_masks,instances.pred_boxes,
                                                                  instances.pred_classes.numpy(), range(1,len(instances))):
                    res_img = np.repeat(pred_mask.numpy()[:, :, None], 3, axis=-1)*img
                    w = int(pred_box[2] - pred_box[0])
                    h = int(pred_box[3] - pred_box[1])
                    y = int(pred_box[1])
                    x = int(pred_box[0])
                    crop_img = res_img[y:y+h, x:x+w]

                    # Save the preprocessed image in the folder dedicated to its class
                    path_pp = os.path.join(path_pp_dataset, img_class, os.path.splitext(img_name)[0] + f"-{i}.jpg")
                    list_pp_paths.append(path_pp)
                    list_pp_classes.append(img_class)
                    list_pp_clothes.append(pred_clothes) # Clothes class predicted by mask-r-cnn

                    cv2.imwrite(path_pp, crop_img)

    # Save the annotation file
    file_string = ""
    for i in range(len(list_pp_paths)):
        file_string += list_pp_paths[i] + "," + list_pp_classes[i] + "," + str(list_pp_clothes[i]) + "\n"

    with open(path_pp_ann, "w") as f:
        f.write(file_string)

Create required folders:

In [ ]:
if not os.path.isdir(path_dir_pp_datasets):
    os.mkdir(path_dir_pp_datasets)

# Train folders
if not os.path.isdir(path_pp_train_dataset):
    os.mkdir(path_pp_train_dataset)
for i in range(n_classes):
    path_dir_class = os.path.join(path_pp_train_dataset, str(i))
    if not os.path.isdir(path_dir_class):
        os.mkdir(path_dir_class)

# Validation folders
if not os.path.isdir(path_pp_valid_dataset):
    os.mkdir(path_pp_valid_dataset)
for i in range(n_classes):
    path_dir_class = os.path.join(path_pp_valid_dataset, str(i))
    if not os.path.isdir(path_dir_class):
        os.mkdir(path_dir_class)

# Test folders
if not os.path.isdir(path_pp_test_dataset):
    os.mkdir(path_pp_test_dataset)
for i in range(n_classes):
    path_dir_class = os.path.join(path_pp_test_dataset, str(i))
    if not os.path.isdir(path_dir_class):
        os.mkdir(path_dir_class)

Patch to make the model work (for some reason sometimes it requires a first manual evaluation, otherwise it raises an error):

In [ ]:
# list_images_names, list_images_classes = read_annotation_file(path_train_ann)
# path_img = os.path.join(path_train_dataset, list_images_names[0])
# img = cv2.imread(path_img)
# plt.imshow(img[:, :, ::-1])
# plt.show()
# res = predictor(img)

Preprocess the training and validation sets:

In [ ]:
print("Preprocessing training set...")
preprocess_train_dataset(path_train_dataset, path_train_ann, path_pp_train_dataset,
                         path_pp_train_ann, path_pp_valid_dataset, path_pp_valid_ann)
print("Done")

Preprocess the test set:

In [ ]:
print("Preprocessing test set...")
preprocess_test_dataset(path_test_dataset, path_test_ann, path_pp_test_dataset,
                        path_pp_test_ann)
print("Done")